## Setting up for curricular experiment

This assumes you have already followed the instructions in `baselines/baseline_t5`, which will set up the baseline clue files for model input

### Datasets
1. Download and unzip the xd cw crossword set from http://xd.saul.pw/xd-clues.zip.
    - Save it as './data/original/xd/clues.tsv'
2. Preprocess the dataset using this notebook
3. The dataset will be saved to k_acw_export_dir (as a single train.json file)
4. We will also produce the anagram dataset


In [10]:
%cd /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/

/Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main


In [11]:
%load_ext autoreload
%autoreload 2

from decrypt.scrape_parse.acw_load import get_clean_xd_clues
from decrypt import config
from decrypt.common.util_data import clue_list_tuple_to_train_split_json
from decrypt.common import validation_tools as vt

k_xd_orig_tsv = config.DataDirs.OriginalData.k_xd_cw        # ./data/original/xd/clues.tsv
k_acw_export_dir = config.DataDirs.DataExport.xd_cw_json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# defaults to strip periods, remove questions, remove abbrevs, remove fillin
stc_map, all_clues = get_clean_xd_clues(k_xd_orig_tsv,
                                        remove_if_not_in_dict=False,
                                        do_filter_dupes=True)
clue_list_tuple_to_train_split_json((all_clues,),
                                    comment='ACW set; xd cw set, all',
                                    export_dir=k_acw_export_dir,
                                    overwrite=False)

INFO:root:loading xd (ACW) set from /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/data/original/xd/clues.tsv
6572827it [00:36, 181559.22it/s]
INFO:root:Counter({'removed_trailing_period': 639298, 'fillin': 395885, 'question word': 177601, 'removed_likely_abbrev': 166457, 'empty': 78866, 'ref': 47562})
INFO:root:Filtered to 5706456 clues


Counter({'removed_trailing_period': 639298, 'fillin': 395885, 'question word': 177601, 'removed_likely_abbrev': 166457, 'empty': 78866, 'ref': 47562})


100%|██████████| 257081/257081 [00:45<00:00, 5620.74it/s] 


removed 3151030 exact dupes
2555426


INFO:root:Counter({1: 2555426})
100%|██████████| 2555426/2555426 [00:06<00:00, 399067.20it/s]
INFO:decrypt.common.util_data:Source target mapping:
	Litigator's group (3) => aba

INFO:decrypt.common.util_data:Finished writing all files


{'idx': 0, 'input': "Litigator's group (3)", 'target': 'aba'}
{'idx': 1, 'input': 'Arab garment (3)', 'target': 'aba'}
{'idx': 2, 'input': 'Arabian garment (3)', 'target': 'aba'}



In [14]:
# produce anagram datasets
# roughly 3 minutes to complete
from decrypt.common import anagrammer
anagrammer.gen_db_with_both_inputs(update_flag="overwrite")

from decrypt.common.util_data import (
    get_anags,
    write_json_tuple
)
import json
import os

INFO:root:Overwriting database at /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/data/generated/anag_db
INFO:root:Adding to db /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/data/generated/anag_db with updateflag overwrite
118619it [00:03, 29801.92it/s]


Counter({1: 118609})
Done.


In [15]:
def make_anag_sets_json():
    all_anags = get_anags(max_num_words=-1)
    json_list = []
    for idx, a_list in enumerate(all_anags):
        json_list.append(dict(idx=idx,
                              anag_list=a_list))
    print(json_list[0])

    # normally would be (idx, input, tgt)
    output_tuple = [json_list,]

    os.makedirs(config.DataDirs.DataExport.anag_dir)
    write_json_tuple(output_tuple,
                     comment="List of all anagram groupings",
                     export_dir=config.DataDirs.DataExport.anag_dir,
                     overwrite=False)

def make_anag_indic_list_json():
    # make the indicator list
    with open(config.DataDirs.OriginalData.k_deits_anagram_list, 'r') as f:
        all_anag_indicators = f.readlines()
        print(len(all_anag_indicators))

    final_indic_list = []
    for a in all_anag_indicators:
        final_indic_list.append(a.replace('_', " ").strip())
    with open(config.DataDirs.DataExport.anag_indics, 'w') as f:
        json.dump(final_indic_list,f)

In [16]:
make_anag_sets_json()

INFO:root:Initializing (non-singleton) Anagrammer from /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/data/generated/anag_db
INFO:root:DONE: Initialized Anagrammer from /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/data/generated/anag_db
100%|██████████| 110341/110341 [00:01<00:00, 75258.13it/s]
INFO:root:Total anagramable: 6543
INFO:decrypt.common.util_data:Finished writing all files


6543
6543
14811
['taint', 'titan']
{'idx': 0, 'anag_list': ['taint', 'titan']}
{'anag_list': ['taint', 'titan'], 'idx': 0}
{'anag_list': ['counter', 'recount', 'trounce'], 'idx': 1}
{'anag_list': ['disburden', 'underbids'], 'idx': 2}



In [17]:
make_anag_indic_list_json()



1160


## Curricular training
1. At this point you should have a files at
 - `./data/clue_json/curricular/ACW/train.json`
 - `./data/clue_json/curricular/anagram/[train.json, anag_indics.json]`

2. Running curricular training is the same as running main t5 vanilla train, except that we pass an extra multitask flag, which specifies the curriculum to use. See `seq2seq/multitask_config`. You should pass one of the names from  `multi_config` dict in that file

For example, to train the naive split with the top performing curricular approach (i.e. the result in table 3 that is ACW + ACW-descramble)
```python
python train_clues.py --default_train=base --name=naive_top_curricular --project=curricular --wandb_dir='./wandb' --data_dir='../data/clue_json/guardian/naive_random' --multitask=ACW__ACW_descramble
```

Note that the modifications on the dataset are done at the

3. To produce Table 3 of the results
    -  we don't need to do a model_eval run since the outputted predictions have 5 generations
       (which is all we report for that table (for faster experimental iteration).
    - we need to run `load_and_run_t5` on all outputs (column 1) and on the anagram subset (column 2)
      See below for how we do this.

4. For our top result in Table 2 (main resuls) we
    1. scale up the curricular period (to 4 total epochs)
```python
python train_clues.py --default_train=base --name=naive_top_curricular --project=curricular --wandb_dir='./wandb' --data_dir='../data/clue_json/guardian/naive_random' --multitask=final_top_result_scaled_up
```
    2. eval with full 100 generations, as before:
e.g., if epoch 10 is best (you'll need to set the run_name)
This runs the eval set (change the run_name)
```python
python train_clues.py --default_val=base --name=curricular_naive_top --project=curricular --data_dir='../data/clue_json/guardian/naive_random' --ckpt_path='./wandb/run_name/files/epoch_10.pth.tar
```


In [19]:
from decrypt.common.label_anagrams import make_label_set

labels = make_label_set()

INFO:decrypt.scrape_parse.guardian_load:Loading splits directly from given json files. Using /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/data/naive_random.json
100%|██████████| 142380/142380 [00:00<00:00, 932101.02it/s]
INFO:decrypt.scrape_parse.guardian_load:Counter({1: 118540, 2: 20105, 3: 2929, 4: 686, 5: 112, 6: 8})
INFO:decrypt.scrape_parse.guardian_load:Clue list length matches Decrypting paper expected length
INFO:decrypt.scrape_parse.guardian_load:Got splits of lenghts [85428, 28476, 28476]
INFO:decrypt.scrape_parse.guardian_load:First three clues of train set:
	[CleanGuardianClue(clue='Suffering to grasp edge of plant', lengths=[8], soln='agrimony', soln_with_spaces='agrimony', idx=-1, dataset='', across_or_down='', pos=(0, 0), unique_clue_id='', type='cryptic', number=0, id='', creator='Chifonie', orig_lengths='8', lengths_punctuation=set()), CleanGuardianClue(clue='Honour Ben and Noel with new order', lengths=[7], soln='ennoble', soln_with_spaces='ennoble', idx=-

AssertionError: 

In [ ]:
# note that this should be run directly on the top model output from curricular training
# otherwise (eg. if 100 beams were used), the top 5 output
# sequences would be expected to change
# remember not to append .json

# eval on the full output (5 beams / 5 sequences)
# this is column 1 of table 3
vt.load_and_run_t5('outputs/model_output.preds',
                   # pre_truncate=5,        # should not be needed since we have only 5 outputs
                   do_length_filter=True)

# run on the anagram subset
# this is column 2 of table 3
vt.load_and_run_t5('outputs/model_output.preds',
                   filter_fcn=vt.make_set_filter(labels, 'anag_direct'),
                   # pre_truncate=5,
                   do_length_filter=True)

# we are looking at agg_top_match (which is after filter)